# ODA data by Recipient
This tutorial shows you how to build a Pandas DataFrame containing ODA data to any number sectors and recipients, from any number of donors, over a range of years.

The tutorial has the following steps:
1. Optionally, download the raw data if you haven't yet done so
2. Import `ODAData`, the main tool used to interact with the data
3. Create an instance of `ODAData` with the specific arguments you would like to use
4. Load the indicators
5. Get the DataFrame
6. Optionally, export the DataFrame

## 1. Download the raw data

The raw data must be downloaded every time you do a fresh installation of the `oda_data` package. The raw data is not distributed with the package given its size, and to make sure users are accessing the latest data when they start working with the package.

Before getting started with analysis for the first time, you must download the raw data. This process can take a while depending on your internet connection, and on the files that you decide to download.

It is highly recommended that you specify the folder where you want to store and read the raw data. This must be done once in each notebook or script where we use the `oda_data` package.

First we will import the necessary functions. In this example, we will only download the CRS data using `download_crs(years)`, but other options include `download_dac1()`, `download_dac2a()`, and `download_multisystem()`.

To reduce file size, we set the small_version parameter to True which removes XXX. You must also set years, as below.

In [1]:
from oda_data import set_data_path

# set the path to the folder where we want to store the data
set_data_path(path="../tutorials")

## 2. Import ODAData

We can gather the data we need using the `ODAData` class. An object from this class can:
    - Get data for specific indicators
        - Optionally, filter the data for specific donors, recipients and years
        - Optionally, exchange and deflate data

You can specify the data path if the raw data has already been downloaded, or if you haven't yet specified the download path in your notebook and script.

In [2]:
from oda_data import ODAData

from oda_data.tools.groupings import recipient_groupings

## 3. Instantiating the ODAData object with the desired arguments

Next, we need to set the right arguments, which are used to create the right DataFrame.
- *years*: you must specify the `years`, as an `int`, `list` or `range`.
- *donors*: you can _optionally_ specify the `donors` you want the output to have (as `int`, or `list`) of donor codes.
- *recipients*: you can _optionally_ specify the `recipients` you want the output to have. Not all indicators need or accept recipients. If using an indicator for which recipients aren't an option, a warning will be logged to the console and the recipients ignored for that indicator.
- *currency*: you can _optionally_ specify the `currency` in which you want your data to be shown. If not specified, by default, `USD` will be used. Other options include `EUR`, `GBP` and `CAN`.
- *prices*: you can _optionally_ specify the `prices` in which you want your data to be shown. If not specified, by default, `current` will be used. The other option is `constant`. If specifying `constant` a `base_year` must be set.
- *base_year*: you must specify a `base_year` if you have set `prices = 'constant'`. If you have chosen `current` prices,
by default, `base_year` will be `None`.


Below are some example settings for this tutorial. For clarity, we will first store them in variables, but you can always pass them directly as arguments to the `ODAData` class.
You can see the available options using the .available_xxx() methods below.

In [ ]:
recipient_groupings().keys()

In [ ]:
recipient_groupings()

In [3]:
# Select years as (for example) a range. Remember ranges are exclusive of the upper bound.
years = range(2012,2022)

# Select donors, which must be specified by their codes. To get all donors, do not use this argument.
donors = 12

# Select recipients. In this example, we have used Developing Countries, total.
recipients = None

# Select the currency. By default 'USD' is shown but we'll get the data in Euros.
currency = 'GBP'

# Select the prices. By default, 'current' is shown, but we'll get the data in constant prices.
prices = 'constant'

# Set the base year. We must set this given that we've asked for constant data.
base_year = 2021

In [ ]:
ODAData().available_donors()

In [ ]:
ODAData().available_recipients()

In [ ]:
ODAData().available_currencies()

In [ ]:
ODAData().available_indicators()

    ## 4. Create an instance of ODAData

In order to get the data we want, we will create an instance of the `ODAData` class using the arguments we specified above.

We will store this instance in a variable called `oda`, which we will use later to load the indicators and get the DataFrame we're after.

In [4]:
oda = ODAData(years=years,
              recipients=recipients,
              donors=donors,
              currency=currency,
              prices=prices,
              base_year=base_year,
              include_names=True)
oda.load_indicator('crs_gender_total_flow_gross')

ODAData(years=range(2012, 2022), donors=[12], recipients=None, currency='GBP', prices='constant', base_year=2021, include_names=True)

In [5]:
df = oda.add_share_of_total(True).get_data()

In [6]:
df.groupby(['year','currency','prices'])['value','share'].sum(numeric_only=True)

C:\Users\MatthewPrice\AppData\Local\Temp\ipykernel_16172\3649604311.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df.groupby(['year','currency','prices'])['value','share'].sum(numeric_only=True)


,,,value,share
year,currency,prices,,
2012,GBP,constant,162.735436,2.75160
2013,GBP,constant,3839.612873,49.90004
2014,GBP,constant,3741.224352,47.97811
2015,GBP,constant,161.267757,1.98984
2016,GBP,constant,1719.350739,19.80790
2017,GBP,constant,4739.160518,53.61882
2018,GBP,constant,269.830609,2.98963
2019,GBP,constant,5515.825377,55.39557
2020,GBP,constant,4938.191725,60.94353


In [ ]:
import oda_data
oda_data.__version__

In [7]:
df.to_csv(r'../tutorials/UK_oda_CRS_gender.csv', index = False)